In [24]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import time # Importing the time library to add a delay before scraping the data
import random # Importing the random library to generate a random delay before scraping the data

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-notifications")  # Disable notifications


browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Navigate to the initial page
browser.get("https://www.nasdaq.com/market-activity/analyst-forecast-change")

# Handle cookie acceptance if present
try:
    accept_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'I Accept')]")))
    accept_button.click()
except TimeoutException:
    print("Accept button not found or unable to click")

# Create an empty list to store the data
data = []
last_page_button = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "button.pagination__page:last-child"))
    )
total_pages = int(last_page_button.get_attribute('data-page'))
#Loop through all pages
for i in range(1, total_pages+1):
    try:
        
        print(f"Processing page {i}")
        
       
        try:
            accept_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'I Accept')]")))
            accept_button.click()
        except TimeoutException:
            print("Accept button not found or unable to click")

        
        page_number = f".pagination__page[data-page='{i}']"
        element = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, page_number)))
        element.click()
        time.sleep(3)
    except TimeoutException:
        print(f"Timeout while waiting for page {i} to load")
        continue  

    
    
    
    rows = browser.find_elements(By.XPATH, "//table[@class='analyst-forecast-change__table']//tbody//tr")
    if i%3 == 0:
        time.sleep(random.randint(2, 5))
    for row in rows:
        try:
            cells = row.find_elements(By.XPATH, ".//td")
            row_data = [cell.text for cell in cells]
            data.append(row_data)
        except StaleElementReferenceException:
            print("Stale Element Exception occurred while processing row. Trying to re-locate the elements.")
            rows = browser.find_elements(By.XPATH, "//table[@class='analyst-forecast-change__table']//tbody//tr")
            break

num_columns = len(data[0])


df_Increase = pd.DataFrame(data, columns=[f'Column{i}' for i in range(1, num_columns + 1)])




browser.quit()

In [22]:
df_Increase.head(100)


,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
0,AGNC,AGNC Investment Corp.,NASDAQ -GS,0.56,0.55,1%,3,1,0
1,ALK,"Alaska Air Group, Inc.",NYSE,-1.09,-1.1,1%,6,0,0
2,AMG,"Affiliated Managers Group, Inc.",NYSE,5.21,5.15,1%,6,3,1
3,AMP,"AMERIPRISE FINANCIAL SERVICES, LLC",NYSE,8.19,8.12,1%,6,3,0
4,AMPH,"Amphastar Pharmaceuticals, Inc.",NASDAQ -GS,0.74,0.73,1%,3,1,0
...,...,...,...,...,...,...,...,...,...
95,WRK,Westrock Company,NYSE,0.25,0.24,3%,4,1,0
96,ANTX,"AN2 Therapeutics, Inc.",NASDAQ -GS,-0.61,-0.63,4%,3,0,0
97,APAM,Artisan Partners Asset Management Inc.,NYSE,0.79,0.76,4%,2,1,0
98,DOOR,Masonite International Corporation,NYSE,2.01,1.93,4%,2,0,0


In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import time # Importing the time library to add a delay before scraping the data
import random # Importing the random library to generate a random delay before scraping the data

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-notifications")  # Disable notifications


browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Navigate to the initial page
browser.get("https://www.nasdaq.com/market-activity/analyst-forecast-change")

# Handle cookie acceptance if present
try:
    accept_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'I Accept')]")))
    accept_button.click()
except TimeoutException:
    print("Accept button not found or unable to click")
WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.table-tabs__tab[data-value='nextFiscalQ']"))).click()
# Create an empty list to store the data
data = []
last_page_button = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "button.pagination__page:last-child"))
    )
total_pages = int(last_page_button.get_attribute('data-page'))
#Loop through all pages
for i in range(1, total_pages+1):
    try:
        
        print(f"Processing page {i}")
        
       
        try:
            accept_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'I Accept')]")))
            accept_button.click()
        except TimeoutException:
            print("Accept button not found or unable to click")

        
        page_number = f".pagination__page[data-page='{i}']"
        element = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, page_number)))
        element.click()
        time.sleep(3)
    except TimeoutException:
        print(f"Timeout while waiting for page {i} to load")
        continue  

    
    
    
    rows = browser.find_elements(By.XPATH, "//table[@class='analyst-forecast-change__table']//tbody//tr")
    if i%3 == 0:
        time.sleep(random.randint(2, 5))
    for row in rows:
        try:
            cells = row.find_elements(By.XPATH, ".//td")
            row_data = [cell.text for cell in cells]
            data.append(row_data)
        except StaleElementReferenceException:
            print("Stale Element Exception occurred while processing row. Trying to re-locate the elements.")
            rows = browser.find_elements(By.XPATH, "//table[@class='analyst-forecast-change__table']//tbody//tr")
            break

num_columns = len(data[0])


df_Increase = pd.DataFrame(data, columns=[f'Column{i}' for i in range(1, num_columns + 1)])




browser.quit()

Processing page 1
Accept button not found or unable to click
Processing page 2
Accept button not found or unable to click
Processing page 3
Accept button not found or unable to click
Processing page 4
Accept button not found or unable to click
Processing page 5
Accept button not found or unable to click
Processing page 6
Accept button not found or unable to click
Processing page 7
Accept button not found or unable to click
Processing page 8
Accept button not found or unable to click
Processing page 9
Accept button not found or unable to click
Processing page 10
Accept button not found or unable to click
Processing page 11
Accept button not found or unable to click
Processing page 12
Accept button not found or unable to click
Processing page 13
Accept button not found or unable to click
Processing page 14
Accept button not found or unable to click
Processing page 15
Accept button not found or unable to click
Processing page 16
Accept button not found or unable to click


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import time # Importing the time library to add a delay before scraping the data
import random # Importing the random library to generate a random delay before scraping the data

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-notifications")  # Disable notifications


browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Navigate to the initial page
browser.get("https://www.nasdaq.com/market-activity/analyst-forecast-change")

# Handle cookie acceptance if present
try:
    accept_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'I Accept')]")))
    accept_button.click()
except TimeoutException:
    print("Accept button not found or unable to click")
WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.table-tabs__tab[data-value='currentFiscalY']"))).click()
# Create an empty list to store the data
data = []
last_page_button = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "button.pagination__page:last-child"))
    )
total_pages = int(last_page_button.get_attribute('data-page'))
#Loop through all pages
for i in range(1, total_pages+1):
    try:
        
        print(f"Processing page {i}")
        
       
        try:
            accept_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'I Accept')]")))
            accept_button.click()
        except TimeoutException:
            print("Accept button not found or unable to click")

        
        page_number = f".pagination__page[data-page='{i}']"
        element = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, page_number)))
        element.click()
        time.sleep(3)
    except TimeoutException:
        print(f"Timeout while waiting for page {i} to load")
        continue  

    
    
    
    rows = browser.find_elements(By.XPATH, "//table[@class='analyst-forecast-change__table']//tbody//tr")
    if i%3 == 0:
        time.sleep(random.randint(2, 5))
    for row in rows:
        try:
            cells = row.find_elements(By.XPATH, ".//td")
            row_data = [cell.text for cell in cells]
            data.append(row_data)
        except StaleElementReferenceException:
            print("Stale Element Exception occurred while processing row. Trying to re-locate the elements.")
            rows = browser.find_elements(By.XPATH, "//table[@class='analyst-forecast-change__table']//tbody//tr")
            break

num_columns = len(data[0])


df_Increase = pd.DataFrame(data, columns=[f'Column{i}' for i in range(1, num_columns + 1)])




browser.quit()

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import time # Importing the time library to add a delay before scraping the data
import random # Importing the random library to generate a random delay before scraping the data

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-notifications")  # Disable notifications


browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Navigate to the initial page
browser.get("https://www.nasdaq.com/market-activity/analyst-forecast-change")

# Handle cookie acceptance if present
try:
    accept_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'I Accept')]")))
    accept_button.click()
except TimeoutException:
    print("Accept button not found or unable to click")
WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.table-tabs__tab[data-value='nextFiscalY']"))).click()
# Create an empty list to store the data
data = []
last_page_button = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "button.pagination__page:last-child"))
    )
total_pages = int(last_page_button.get_attribute('data-page'))
#Loop through all pages
for i in range(1, total_pages+1):
    try:
        
        print(f"Processing page {i}")
        
       
        try:
            accept_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'I Accept')]")))
            accept_button.click()
        except TimeoutException:
            print("Accept button not found or unable to click")

        
        page_number = f".pagination__page[data-page='{i}']"
        element = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, page_number)))
        element.click()
        time.sleep(3)
    except TimeoutException:
        print(f"Timeout while waiting for page {i} to load")
        continue  

    
    
    
    rows = browser.find_elements(By.XPATH, "//table[@class='analyst-forecast-change__table']//tbody//tr")
    if i%3 == 0:
        time.sleep(random.randint(2, 5))
    for row in rows:
        try:
            cells = row.find_elements(By.XPATH, ".//td")
            row_data = [cell.text for cell in cells]
            data.append(row_data)
        except StaleElementReferenceException:
            print("Stale Element Exception occurred while processing row. Trying to re-locate the elements.")
            rows = browser.find_elements(By.XPATH, "//table[@class='analyst-forecast-change__table']//tbody//tr")
            break

num_columns = len(data[0])


df_Increase = pd.DataFrame(data, columns=[f'Column{i}' for i in range(1, num_columns + 1)])




browser.quit()

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from bs4 import BeautifulSoup

driver = webdriver.Chrome()

driver.get("https://www.nasdaq.com/market-activity/stocks/agnc/analyst-research")
try:
    accept_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'I Accept')]")))
    accept_button.click()
except TimeoutException:
    print("Accept button not found or unable to click")

html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')

aside_content = soup.find('aside', class_='quote-subdetail__about-market')

if aside_content:
    print("Content within the <aside> tag:")
    print(aside_content.get_text(strip=True))
else:
    print("Content not found within the specified <aside> tag.")

# Now, let's print the entire HTML page source
print("\n\nFull HTML Page Source:")
print(html)

driver.quit()  # Don't forget to quit the driver when done